In [4]:
############################################## begin replaying the code ##################################################
# https://www.kaggle.com/code/steubk/arxiv-taxonomy-e-top-influential-papers/notebook

# we will do the same as the above kaggle project, but need to make some change to accommodate some missing files or big files


# Notes

## **Purpose:**
- separate the data arxiv-metadata-oai-snapshot.json into multiple files, categorized by year, and extract information from arxiv-metadata-oai-snapshot.json and save to different .csv files
- extract information from internal-references-pdftotext.json and save it to a .csv file

## **Data File:**
1. The metadata comes from https://www.kaggle.com/datasets/Cornell-University/arxiv. It contains the latest data till now.

2. Internal reference data comes from https://github.com/mattbierbaum/arxiv-public-datasets/releases. The file is https://github.com/mattbierbaum/arxiv-public-datasets/releases/download/v0.2.0/internal-references-v0.2.0-2019-03-01.json.gz


*---------Please note-----------*

The reference code from project "arXiv - Taxonomy e Top Influential Papers" ( https://www.kaggle.com/code/steubk/arxiv-taxonomy-e-top-influential-papers/input?select=arxiv-metadata-oai-snapshot-2020-08-14.json) is using the same dataset, but the dataset version is earlier "arxiv-metadata-oai-snapshot-2020-08-14.json", so the generated result is different because the up-to-date dataset contains data beyond 2020.*




In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **How to Run:**
- There are two parts in this notebook
- First part is used to extract info from the two huge json files. All the generated .csv files will be put into subfolder "data".
    - Please run the cells in sequence.
    - **When the code requires to upload Kaggle profile, upload your Kaggle profile.(cell 3)** This profile will be stored in colab session temporarily, so it will not get shared with others once you exit your colab session.
    - **Please mount your google drive by running the corresponding cell below (cell 4)**. Otherwise, the generated data file could not get saved.
    - **please specify the variable project_path(cell 5)** to point to your google drive data location in colab. Other variables such as project_data_path, metadata_file, ref_file is specified already, and you do not need to change them.

- second part is used to do data analysis.
- part 1 and part 2 are independent. That means, once you finish running part 1, you could shut down your notebook. The next time it is opened, you could start running from part 2.

- Please refer to https://www.kaggle.com/code/steubk/arxiv-taxonomy-e-top-influential-papers/notebook when necessary. Pls note that some steps are updated to accomodate the limited momery and CPU resource.

In [6]:
!pip install requests
!pip install wordcloud
!pip install -q kaggle

In [7]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

import requests
from bs4 import BeautifulSoup
import re

import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud, STOPWORDS
import zipfile
from google.colab import files
import gzip

In [ ]:
# take a look at this post on how to download data from Kaggle to Colab
# https://www.kaggle.com/discussions/general/74235

# You could use the following steps, but your Kaggle profile should be uploaded when prompted

display("upload your Kaggle profile file now:")

files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

display("Start getting the data file from Kaggle...")

# How to get the dataset address: copy API command to download the dataset: go to the top-right corner three-points button, and click it -> choose "Copy API Command" then copy it here, not forget to add ! at the beginning
!kaggle datasets download -d Cornell-University/arxiv
zip_ref = zipfile.ZipFile('arxiv.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

display("Finish getting the data file and extracting it from Kaggle.")
display("Start getting the data file from github...")

!wget https://github.com/mattbierbaum/arxiv-public-datasets/releases/download/v0.2.0/internal-references-v0.2.0-2019-03-01.json.gz

display("Finish getting the data files from Kaggle and Github. Both data files are stored in session under /content")


'upload your Kaggle profile file now:'

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# this step is necessary if you want to save the transformed data into google drive and reuse.
from google.colab import drive
drive.mount("/content/drive")


# Part 1 - preprocessing the data

In [ ]:
project_path = r"/content/drive/MyDrive/Colab Notebooks/nus-cs5246-master-text-mining/final_project"
project_data_path = project_path + "/data"
if not os.path.exists(project_path + "/data"):
    os.makedirs(project_path + '/data/')

metadata_file = "/content/arxiv-metadata-oai-snapshot.json"
ref_file = "/content/internal-references-v0.2.0-2019-03-01.json.gz"

In [ ]:

with gzip.open(ref_file, "r") as f:
    citations = json.load(f)

with open(project_data_path + "/arxiv-metadata-ext-citation.csv","w+") as f_out :
    f_out.write("id,id_reference\n")
    for i,id in enumerate(citations):
        if i % 50000 == 0:
            print("Processing the citation records from {} to {}".format(i, i+ 50000))

        for k in citations[id]:
            f_out.write(f'{id},{k}\n')

print("Finish processing all the internal citation and save it to {}".format("data/arxiv-metadata-ext-citation.csv"))

# uncomment the below code if you want to check the data
# df_citations = pd.read_csv(project_data_path + "/arxiv-metadata-ext-citation.csv",dtype={"id":object,"id_reference":object})
# df_citations.head()
# print("total records of df:citations:", df_citations.shape)

In [ ]:
## reorganize the format of the categories for each record

with open(project_data_path + "/arxiv-metadata-ext-category.csv","w+") as f_out :
    f_out.write("id,category_id\n")

    with open(metadata_file) as f_in:
        for i,line in enumerate(f_in):
            if i % 100000 == 0:
                print("Processing the categories of records from {} to {}...".format(i, i + 100000))

            row = json.loads(line)
            id = row["id"]
            categories = row["categories"].split()
            for c in categories:
                f_out.write ( f'"{id}","{c}"\n'  )
print("Finish processing the categories of records and save it to {}".format(project_data_path + "/arxiv-metadata-ext-category.csv"))

# uncomment the below code if you want to check the data
# df_categories = pd.read_csv("data/arxiv-metadata-ext-category.csv",dtype={"id":object,"category_id":object})
# df_categories.head()
# print("total records of df:citations:", df_categories.shape)

In [ ]:
#################    DO NOT RUN THIS CELL, just For Your Reference     ##############################
################ This cell includes the original method  on Keggle to organize ######################
################ meta data, but the info is too huge and it could not save to  ######################
################                        csv file.                              ######################
## create a metadata paper csv
## This cell is just for reference. If we combine all the data into one csv file, we will run out of memory. We need to separate
## the files by year. See the next cell and this cell's result could be deleted.


# METADATA_PATH = r'archive/arxiv-metadata-oai-snapshot.json'


# titles = []
# abstracts = []
# ids = []
# authors = []
# journal_refs = []
# licenses = []


# with open(METADATA_PATH) as f_in:
#     for i,line in enumerate(f_in):
#         if i % 50000 == 0:
#             print("Start processing record from {} to {}...".format(i, i+50000))

#         row = json.loads(line)

#         titles.append(row["title"])
#         abstracts.append(row["abstract"])
#         ids.append(row["id"])
#         authors.append(row["authors"])
#         journal_refs.append(row["journal-ref"])
#         licenses.append(row["license"])

#     print("Finish processing all the papers. Total records: {}".format(i))

# # the dataframe is too big and could not save to csv
# df_papers = pd.DataFrame({
#     'id' : ids,
#     'title' : titles,
#     'abstract' : abstracts,
#     'authors' : authors,
#     'journal-ref' : journal_refs,
#     'license':licenses

# })
# df_papers.to_csv("arxiv-metadata-ext-paper.csv", index=False)
# df_papers.head()

In [ ]:
###### separate the original meta file to different files by year
# This cell just creates two functions to save to .csv or .xlsx

def saveToFile(year_value, file_name, ids, titles, abstracts, authors, journal_refs, years, licenses):
    df_papers = pd.DataFrame({
    'id' : ids[year_value],
    'title' : titles[year_value],
    'abstract' : abstracts[year_value],
    'authors' : authors[year_value],
    'journal-ref' : journal_refs[year_value],
    'year': years[year_value],
    'license':licenses[year_value]})

    df_papers.to_csv(file_name, index=False)

def saveToExcelFile(year_value, file_name, ids, titles, abstracts, authors, journal_refs, years, licenses):
    df_papers = pd.DataFrame({
    'id' : ids[year_value],
    'title' : titles[year_value],
    'abstract' : abstracts[year_value],
    'authors' : authors[year_value],
    'journal-ref' : journal_refs[year_value],
    'year': years[year_value],
    'license':licenses[year_value]})

    df_papers.to_excel(file_name)

In [ ]:
## save metadata data and then save them to different .csv

import json
import pandas as pd

def get_metadata():
    with open(metadata_file, 'r') as f:
        for line in f:
            yield line

# create a dictionary structure for each piece of meta data and later on, we could save the value by year
titles = {}
abstracts = {}
ids = {}
authors = {}
journal_refs = {}
licenses = {}
years = {}
for year in range(1999, 2025):
    titles[year] = []
    abstracts[year] = []
    ids[year] = []
    authors[year] = []
    journal_refs[year] = []
    licenses[year] = []
    years[year] = []

index = 0
records_21st = 0
metadata = get_metadata()

print("Start processing the meta data...\n")

for paper in metadata:
    index += 1
    # each one item is a paper info.
    paper_dict = json.loads(paper)

    try:
        year = int(paper_dict['versions'][0]['created'].split()[3])

        if year < 2000:
            year = 1999
        else:
            records_21st +=1

        years[year].append(year)
        titles[year].append(paper_dict["title"])
        abstracts[year].append(paper_dict["abstract"])
        ids[year].append(paper_dict["id"])
        authors[year].append(paper_dict["authors"])
        journal_refs[year].append(paper_dict["journal-ref"])
        licenses[year].append(paper_dict["license"])
    except:
        print("error when handling paper id {}".format(paper_dict["id"]))
        print(paper_dict)
        # break

    if index % 50000 == 0:
        print("Start processing index {} to {}...".format(index, index+50000))

print("Finish processing the papers. Totally there are {} records. {} records are after 2000.".format(index, records_21st))

######## This step is necessary, because we need to open those csv files in the later steps ##########
print("Now save to files by year...")

for i in range(1999, 2025):
    if len(ids) > 0:
        saveToFile(i, project_data_path + "/output_" + str(i) + ".csv", ids, titles, abstracts, authors, journal_refs, years, licenses)

        if i == 1999:
            print("Finish saving to the file {}.".format(project_data_path + "/output_" + str(i) + ".csv for all the records before 2000."))
        else:
            print("Finish saving to the file {}.".format(project_data_path + "/output_" + str(i) + ".csv"))

print("Finish saving all the files.")

In [ ]:
######################### optional ###################################################################
########### if you want to check data more conviently with excel file format, you could run ##########
for i in range(2001, 2025):
    print("Start saving to the file {}.".format(project_data_path + "/output_" + str(i) + ".xlsx"))
    saveToExcelFile(i, "data/output_" + str(i) + ".xlsx", ids, titles, abstracts, authors, journal_refs, years, licenses)
print("Finish saving all the files to xlsx.")

In [ ]:

######## This cell will save each paper's version 1 info to a file. It's running a bit slow...

## create a metadata version csv
with open(project_data_path + "/arxiv-metadata-ext-version.csv","w+") as f_out :
    f_out.write("id,year,month\n")

    with open(metadata_file) as f_in:
        for i,line in enumerate(f_in):
            if i % 100000 == 0:
                print("Start processing the version of records from {} to {}".format(i, i + 100000))


            row = json.loads(line)
            id = row["id"]
            date_value = pd.to_datetime(row["versions"][0]['created'])
            month = date_value.month
            year = date_value.year

            f_out.write (f'{id},{year},{month}\n')

print("Finish processing the versions of records and save it to {}".format(project_data_path + "/arxiv-metadata-ext-version.csv"))


In [ ]:
## load taxonomy from https://arxiv.org/category_taxonomy
website_url = requests.get('https://arxiv.org/category_taxonomy').text
soup = BeautifulSoup(website_url,'lxml')

root = soup.find('div',{'id':'category_taxonomy_list'})

tags = root.find_all(["h2","h3","h4","p"], recursive=True)

level_1_name = ""
level_2_code = ""
level_2_name = ""

level_1_names = []
level_2_codes = []
level_2_names = []
level_3_codes = []
level_3_names = []
level_3_notes = []

for t in tags:
    if t.name == "h2":
        level_1_name = t.text
        level_2_code = t.text
        level_2_name = t.text
    elif t.name == "h3":
        raw = t.text
        level_2_code = re.sub(r"(.*)\((.*)\)",r"\2",raw)
        level_2_name = re.sub(r"(.*)\((.*)\)",r"\1",raw)
    elif t.name == "h4":
        raw = t.text
        level_3_code = re.sub(r"(.*) \((.*)\)",r"\1",raw)
        level_3_name = re.sub(r"(.*) \((.*)\)",r"\2",raw)
    elif t.name == "p":
        notes = t.text
        level_1_names.append(level_1_name)
        level_2_names.append(level_2_name)
        level_2_codes.append(level_2_code)
        level_3_names.append(level_3_name)
        level_3_codes.append(level_3_code)
        level_3_notes.append(notes)

df_taxonomy = pd.DataFrame({
    'group_name' : level_1_names,
    'archive_name' : level_2_names,
    'archive_id' : level_2_codes,
    'category_name' : level_3_names,
    'category_id' : level_3_codes,
    'category_description': level_3_notes
})
df_taxonomy.to_csv(project_data_path + "/arxiv-metadata-ext-taxonomy.csv", index=False)
print("Finish processing the taxonomy of records and save it to {}".format(project_data_path + "/arxiv-metadata-ext-taxonomy.csv"))

df_taxonomy.groupby(["group_name","archive_name"]).head(3)


# Part 2
*************************************************************************************************
**Till this point, we have finished pre-processing data and store them under /data subfolder.**

* The following steps are independent from the above steps *
*************************************************************************************************

In [ ]:
######################### start to read from the pre-processed documents ##############################

df_citations = pd.read_csv(project_data_path + "/arxiv-metadata-ext-citation.csv",dtype={"id":object,"id_reference":object})
df_citations.head()
print("total records of df:citations:", df_citations.shape)

df_categories = pd.read_csv(project_data_path + "/arxiv-metadata-ext-category.csv",dtype={"id":object,"category_id":object})
df_categories.head()
print("total records of df:categories:", df_categories.shape)

df_taxonomy = pd.read_csv(project_data_path + "/arxiv-metadata-ext-taxonomy.csv")
df_taxonomy.head()
print("total records of df_taxonomy:",df_taxonomy.shape)

df_versions = pd.read_csv(project_data_path + "/arxiv-metadata-ext-version.csv", dtype={'id': object})
df_versions.head()
print("total records of df_versions:",df_versions.shape)



In [ ]:
###### this cell defines some functions

def count_by_category_and_year(group_name):
    cats = df_categories.merge(df_taxonomy, on="category_id").query("group_name == @group_name").merge(df_versions[["id","year"]], on ="id")
    cats = cats.groupby(["year","category_name"]).count().reset_index().pivot(index="category_name", columns="year",values="id")
    return cats

def count_by_archive_and_year(group_name):
    cats = df_categories.merge(df_taxonomy, on="category_id").query("group_name == @group_name").merge(df_versions[["id","year"]], on ="id")
    cats = cats.groupby(["year","archive_name"]).count().reset_index().pivot(index="archive_name", columns="year",values="id")
    return cats

def show_count_by_category_and_year(group_name,figsize=(20,10)):
    plt.figure(figsize=figsize)
    plt.title(f"{group_name} papers by category and year")
    sns.heatmap(count_by_category_and_year(group_name),cmap="Greens", linewidths=0.01, linecolor='palegreen')
    plt.show()

def show_count_by_archive_and_year(group_name="Physics",figsize=(10,5)):
    plt.figure(figsize=figsize)
    plt.title(f"{group_name} papers by archive and year")
    sns.heatmap(count_by_archive_and_year(group_name),cmap="Greens", linewidths=0.01, linecolor='palegreen')
    plt.show()

    # updated from the original Kaggle code
def top_k_influential (group_name, top_k=5, threshold=100):
    print("Calculating top influential papers. Take some time ...")
    # NOTE: the current algorithm will put one paper into one or more categories. Under this condition, one citation of a paper
    # which falls into 1+ categories will be calculated multiple times!!! It is designed like this
    ids = df_categories.merge(df_taxonomy, on="category_id").query("group_name ==@group_name").drop_duplicates(["id","group_name"], inplace=False)["id"].values
    cits = df_citations.query('id.isin(@ids)', engine="python").merge(df_versions[["id","year"]], on ="id").groupby(["year","id_reference"]).count()
    cits = cits.reset_index()
    cits.columns = ['year', 'id', 'citation_count']
    cits = cits.loc[cits.groupby('year')['citation_count'].nlargest(top_k).reset_index()['level_1']]
    cits = cits.query("citation_count > @threshold")
    return cits


    # newly added function
def get_paper_info(cits):
    id_ls = cits['id'].values
    meta_df = pd.DataFrame()
    for year in range(1999, 2022):
        df = pd.read_csv(project_data_path + "/output_" + str(year) + ".csv", dtype={"id":object})
        print("Check {} data which contains the paper id...".format(year))
        if year == 1999:
            meta_df = df[df['id'].isin(id_ls)]
        else:
            meta_df = pd.concat([meta_df, df[df['id'].isin(id_ls)]], axis = 0)

    cits = cits.merge(meta_df, how='left', on='id')
    cits = cits.rename(columns={"year_x":"citation_year", "year_y": "first_publish_year"})
    return cits

    # updated from the original Kaggle code
def show_influential_heatmap (group_name, cits, figsize=(8,6)):
    # we have two years related to the paper. Its citation year and its publishing year.
    # at this moment, we use its citation_year to form heatmap
    # we have changed id_reference to id in function top_k_influential() so Values = "id"
    hm_cits =  cits.pivot(index="title", columns="citation_year",values="citation_count")

    plt.figure(figsize=figsize)
    plt.title("Top influential papers by year")
    sns.heatmap(hm_cits,cmap="Greens", linewidths=0.01, linecolor='palegreen')
    plt.show()

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

def show_influential_table ( cits):
    df = cits.groupby(["id","title","authors","abstract"]).agg({"citation_count":"sum"}).reset_index()
    df = df.sort_values(by="citation_count",ascending = False).reset_index(drop=True)
    df ["url"] = df["id"].map(lambda x:  f'https://arxiv.org/pdf/{x}' )
    df ["authors"] = df["authors"].map(lambda x: x if len(str(x)) < 50 else str(x)[:47] + "..." )

    df =df [["title","authors","abstract","url","citation_count"]]
    return df.style.format({'url': make_clickable})

# Thanks : https://www.kaggle.com/aashita/word-clouds-of-various-shapes ##
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0),
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'We', 'paper', 'new'}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color='black',
                    stopwords = stopwords,
                    max_words = max_words,
                        max_font_size = max_font_size,
                    random_state = 42,
                    width=800,
                    height=400,
                    mask = mask,
                    min_word_length = 4,
                    #normalize_plurals = True,
                    #collocations = True,
                       #collocation_threshold = 10
                         )
    wordcloud.generate(str(text))

    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = wordcloud.ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,
                                  'verticalalignment': 'bottom'})
    else:

        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black',
                                  'verticalalignment': 'bottom'})
        plt.axis('off');
        plt.tight_layout()

In [ ]:

############## Copy from the Keggle project ###############
# check the publication per monthly
df = df_versions.groupby(["year","month"]).agg({"id":'count'}).reset_index()
df["tot"] = df["id"].cumsum()

df = df.query("year > 2000 and ( year < 2025)")
df["month"] =  df["year"].astype(str) + "-" + df["month"].astype(str)
display(df.head())

# group by group_name and calculate the total paper amount per group
_df = df_categories.merge(df_taxonomy, on="category_id").drop_duplicates(["id","group_name"]).groupby("group_name").agg({"id":"count"}).sort_values(by="id",ascending=False).reset_index()
# id and category_id is different. id refers to paper's unique id, category_id is a category
_df.columns = ['group_name', 'paper_amount']
display(_df.head())


In [ ]:
#### copy Kaggle code. Visualize the data

fig = plt.figure(figsize=(20,10))
ax1 = plt.subplot2grid((2, 2), (0, 0))
ax1.title.set_text('ArXiv papers')
ax1.plot(df["month"], df["tot"])
ax1.hlines(y=1e6, xmin=0, xmax=len(df), color='green', linestyle="dotted")
ax1.hlines(y=1.5e6, xmin=0, xmax=len(df), color='green', linestyle="dotted")
ax1.set_xticks(np.arange(0, len(df), 12.0))
ax1.tick_params('x',labelrotation=90)

ax2 = plt.subplot2grid((2, 2), (1, 0))
ax2.title.set_text("ArXiv papers by month")
ax2.plot(df["month"], df["id"])
ax2.hlines(y=10000, xmin=0, xmax=len(df), color='green', linestyle="dotted")
ax2.hlines(y=15000, xmin=0, xmax=len(df), color='green', linestyle="dotted")
ax2.set_xticks(np.arange(0, len(df), 12.0))
ax2.tick_params('x',labelrotation=90)

ax3 = plt.subplot2grid((2, 2), (0, 1), rowspan=2)
ax3.title.set_text("ArXiv papers by group")
explode = (0, 0, 0, 0.2, 0.3, 0.3, 0.2, 0.1)
ax3.pie(_df["paper_amount"],  labels=_df["group_name"], autopct='%1.1f%%', startangle=160, explode=explode)


plt.tight_layout()
plt.show()

In [ ]:

# Claudia: copy Kaggle data visualization. Change something here. df_versions dataframe only contains version 1 data, so no need to store field "version"
cats = df_categories.merge(df_taxonomy, on="category_id").merge(df_versions[["id","year"]], on ="id")
cats = cats.groupby(["year","group_name"]).count().reset_index().pivot(index="group_name", columns="year",values="id")

# Claudia: heatmap shows the relationship between year, group_name and paper publishing amount
plt.figure(figsize=(10,5))
plt.title("ArXiv papers by group and year")
sns.heatmap(cats,cmap="Greens", linewidths=0.01, linecolor='palegreen')
plt.show()

In [ ]:
# Kaggle data visualization, show year, category and publishing amount relationship
group_name="Physics"
show_count_by_archive_and_year(group_name)

# for Computer Science, show_count_by_category_and_year because at archive level, there is no diffence
group_name="Computer Science"
show_count_by_category_and_year(group_name)


In [ ]:
# Kaggle data visualization. Show the top k influential papers. it uses the citation number as the
# measurement. Pls refer to method top_k_influential
group_name = "Physics"
cits = top_k_influential (group_name, top_k=3)
cits = get_paper_info(cits)
show_influential_heatmap (group_name, cits=cits, figsize=(8,6))
show_influential_table (cits)

In [ ]:
group_name = "Computer Science"
cits = top_k_influential (group_name, top_k=5)
cits = get_paper_info(cits)
show_influential_heatmap (group_name, cits=cits, figsize=(6,4))
show_influential_table (cits)

In [ ]:
group_name="Electrical Engineering and Systems Science"

show_count_by_category_and_year (group_name,figsize=(8,6))
cits = top_k_influential (group_name, top_k=1, threshold=1)
cits = get_paper_info(cits)
show_influential_heatmap (group_name, cits=cits, figsize=(6,4))
show_influential_table (cits)

In [ ]:
group_name="Quantitative Finance"

show_count_by_category_and_year (group_name,figsize=(10,4))
cits = top_k_influential (group_name, top_k=1, threshold=3)
cits = get_paper_info(cits)
show_influential_heatmap (group_name, cits=cits, figsize=(10,3))

In [ ]:
df_specialCase = df_citations.query("id_reference == '1412.6980'")\
.merge(df_categories,on="id")\
.merge(df_taxonomy,on="category_id").drop_duplicates(["id","group_name"])\
.merge(df_versions[["id","year"]], on ="id")

hmap =df_specialCase.groupby(["group_name","year"]).agg({"id":"count"}).reset_index().pivot(index=["group_name"], columns="year",values="id")

plt.figure(figsize=(10,5))
plt.title("Papers that reference 'Adam: A Method for Stochastic Optimization'")
sns.heatmap(hmap,cmap="Greens", linewidths=0.01, linecolor='palegreen', annot=True, fmt=".0f")
plt.show()